In [ ]:
import os
import time
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime
import random

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.transforms import functional as TF

import cv2
from PIL import Image
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                             f1_score, confusion_matrix, roc_auc_score, 
                             roc_curve, classification_report)
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Set seeds for reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# Check GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Number of GPUs: {torch.cuda.device_count()}")

In [ ]:
CONFIG = {
    # Data paths
    'data_dir': '/kaggle/input/140k-real-and-fake-faces',
    'output_dir': '/kaggle/working',
    
    # Model parameters
    'img_size': 224,
    'batch_size': 64,
    'num_epochs': 30,
    'learning_rate': 0.0001,
    'weight_decay': 1e-4,
    
    # Training parameters
    'num_workers': 4,
    'early_stopping_patience': 7,
    'reduce_lr_patience': 3,
    'reduce_lr_factor': 0.5,
    
    # Data augmentation
    'rotation_degrees': 10,
    'train_split': 0.8,
    'val_split': 0.1,
    # test_split will be 0.1
    
    # Seed
    'seed': 42
}

print("Hyperparameter Configuration:")
print(json.dumps(CONFIG, indent=2))

In [ ]:
class FacePreprocessor:
    """Detect, crop, and preprocess faces from images"""
    
    def __init__(self):
        # Load face cascade classifier
        cascade_path = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
        self.face_cascade = cv2.CascadeClassifier(cascade_path)
    
    def detect_and_crop_face(self, image):
        """Detect face and crop to face region"""
        # Convert PIL to numpy array
        img_array = np.array(image)
        
        # Convert to grayscale for face detection
        gray = cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY)
        
        # Detect faces
        faces = self.face_cascade.detectMultiScale(
            gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30)
        )
        
        # If face detected, crop to largest face
        if len(faces) > 0:
            # Get largest face
            largest_face = max(faces, key=lambda x: x[2] * x[3])
            x, y, w, h = largest_face
            
            # Add padding (10%)
            padding = int(0.1 * max(w, h))
            x = max(0, x - padding)
            y = max(0, y - padding)
            w = w + 2 * padding
            h = h + 2 * padding
            
            # Crop face
            face_crop = img_array[y:y+h, x:x+w]
            return Image.fromarray(face_crop)
        
        # If no face detected, return original
        return image

In [ ]:
class FaceDataset(Dataset):
    """Custom dataset for face classification with preprocessing"""
    
    def __init__(self, image_paths, labels, transform=None, preprocessor=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform
        self.preprocessor = preprocessor
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        # Load image
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        
        # Preprocess (crop face)
        if self.preprocessor:
            image = self.preprocessor.detect_and_crop_face(image)
        
        # Apply transforms
        if self.transform:
            image = self.transform(image)
        
        label = self.labels[idx]
        return image, label

In [ ]:
def get_transforms(mode='train', img_size=224, rotation_degrees=10):
    """Get data transforms for training/validation/test"""
    
    if mode == 'train':
        return transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.RandomRotation(degrees=rotation_degrees),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                               std=[0.229, 0.224, 0.225])
        ])
    else:
        return transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                               std=[0.229, 0.224, 0.225])
        ])


In [ ]:
def load_dataset(data_dir, train_split=0.8, val_split=0.1):
    """Load and split dataset into train/val/test"""
    
    print("Loading dataset...")
    
    # Get paths
    real_dir = Path(data_dir) / 'real_vs_fake' / 'real-vs-fake' / 'train' / 'real'
    fake_dir = Path(data_dir) / 'real_vs_fake' / 'real-vs-fake' / 'train' / 'fake'
    
    # Collect all image paths
    real_images = list(real_dir.glob('*.jpg')) + list(real_dir.glob('*.png'))
    fake_images = list(fake_dir.glob('*.jpg')) + list(fake_dir.glob('*.png'))
    
    print(f"Found {len(real_images)} real images")
    print(f"Found {len(fake_images)} fake images")
    
    # Create labels (0: real, 1: fake)
    real_labels = [0] * len(real_images)
    fake_labels = [1] * len(fake_images)
    
    # Combine
    all_images = real_images + fake_images
    all_labels = real_labels + fake_labels
    
    # Shuffle
    combined = list(zip(all_images, all_labels))
    random.shuffle(combined)
    all_images, all_labels = zip(*combined)
    all_images = list(all_images)
    all_labels = list(all_labels)
    
    # Split dataset
    n_total = len(all_images)
    n_train = int(train_split * n_total)
    n_val = int(val_split * n_total)
    
    train_images = all_images[:n_train]
    train_labels = all_labels[:n_train]
    
    val_images = all_images[n_train:n_train+n_val]
    val_labels = all_labels[n_train:n_train+n_val]
    
    test_images = all_images[n_train+n_val:]
    test_labels = all_labels[n_train+n_val:]
    
    print(f"\nDataset split:")
    print(f"Train: {len(train_images)} images")
    print(f"Val: {len(val_images)} images")
    print(f"Test: {len(test_images)} images")
    
    return (train_images, train_labels), (val_images, val_labels), (test_images, test_labels)

# Load dataset
train_data, val_data, test_data = load_dataset(
    CONFIG['data_dir'], 
    CONFIG['train_split'], 
    CONFIG['val_split']
)

In [ ]:
print("\nCreating dataloaders...")

preprocessor = FacePreprocessor()

train_dataset = FaceDataset(
    train_data[0], train_data[1],
    transform=get_transforms('train', CONFIG['img_size'], CONFIG['rotation_degrees']),
    preprocessor=preprocessor
)

val_dataset = FaceDataset(
    val_data[0], val_data[1],
    transform=get_transforms('val', CONFIG['img_size']),
    preprocessor=preprocessor
)

test_dataset = FaceDataset(
    test_data[0], test_data[1],
    transform=get_transforms('test', CONFIG['img_size']),
    preprocessor=preprocessor
)

train_loader = DataLoader(
    train_dataset, 
    batch_size=CONFIG['batch_size'],
    shuffle=True,
    num_workers=CONFIG['num_workers'],
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=CONFIG['batch_size'],
    shuffle=False,
    num_workers=CONFIG['num_workers'],
    pin_memory=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=CONFIG['batch_size'],
    shuffle=False,
    num_workers=CONFIG['num_workers'],
    pin_memory=True
)

print(f"Train batches: {len(train_loader)}")
print(f"Val batches: {len(val_loader)}")
print(f"Test batches: {len(test_loader)}")


In [ ]:
class FaceClassifierCNN(nn.Module):
    """
    Custom CNN for face classification
    Architecture: 5 convolutional blocks + 3 fully connected layers
    """
    
    def __init__(self, num_classes=2, dropout_rate=0.5):
        super(FaceClassifierCNN, self).__init__()
        
        # Convolutional Block 1
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(0.2)
        )
        
        # Convolutional Block 2
        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(0.3)
        )
        
        # Convolutional Block 3
        self.conv3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(0.4)
        )
        
        # Convolutional Block 4
        self.conv4 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(0.4)
        )
        
        # Convolutional Block 5
        self.conv5 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(0.5)
        )
        
        # Global Average Pooling
        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        
        # Fully Connected Layers
        self.fc = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),
            nn.Linear(256, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),
            nn.Linear(128, num_classes)
        )
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.global_avg_pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# Initialize model
model = FaceClassifierCNN(num_classes=2, dropout_rate=0.5)
model = model.to(device)

# Multi-GPU support
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs")
    model = nn.DataParallel(model)

print("\nModel Architecture:")
print(model)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\nTotal parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")


In [ ]:
# Loss function: Cross Entropy Loss
criterion = nn.CrossEntropyLoss()

# Optimizer: Adam with weight decay
optimizer = optim.Adam(
    model.parameters(),
    lr=CONFIG['learning_rate'],
    weight_decay=CONFIG['weight_decay']
)

# Learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='min',
    factor=CONFIG['reduce_lr_factor'],
    patience=CONFIG['reduce_lr_patience'],
    verbose=True
)

print("\nObjective and Loss Functions:")
print(f"Loss Function: Cross Entropy Loss")
print(f"Optimizer: Adam (lr={CONFIG['learning_rate']}, weight_decay={CONFIG['weight_decay']})")
print(f"Scheduler: ReduceLROnPlateau (patience={CONFIG['reduce_lr_patience']}, factor={CONFIG['reduce_lr_factor']})")


In [ ]:
def train_epoch(model, train_loader, criterion, optimizer, device):
    """Train for one epoch"""
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    pbar = tqdm(train_loader, desc='Training')
    for inputs, labels in pbar:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        pbar.set_postfix({
            'loss': f'{loss.item():.4f}',
            'acc': f'{100.*correct/total:.2f}%'
        })
    
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100. * correct / total
    return epoch_loss, epoch_acc

def validate(model, val_loader, criterion, device):
    """Validate the model"""
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    all_probs = []
    
    with torch.no_grad():
        for inputs, labels in tqdm(val_loader, desc='Validation'):
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            probs = torch.softmax(outputs, dim=1)
            _, predicted = outputs.max(1)
            
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs[:, 1].cpu().numpy())
    
    epoch_loss = running_loss / len(val_loader)
    epoch_acc = 100. * correct / total
    
    return epoch_loss, epoch_acc, all_preds, all_labels, all_probs


In [ ]:
print("\n" + "="*80)
print("STARTING TRAINING")
print("="*80)

# Initialize logging
history = {
    'train_loss': [],
    'train_acc': [],
    'val_loss': [],
    'val_acc': [],
    'learning_rates': []
}

best_val_loss = float('inf')
best_val_acc = 0.0
patience_counter = 0
start_time = time.time()

# Training loop
for epoch in range(CONFIG['num_epochs']):
    epoch_start_time = time.time()
    
    print(f"\nEpoch {epoch+1}/{CONFIG['num_epochs']}")
    print("-" * 80)
    
    # Train
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    
    # Validate
    val_loss, val_acc, val_preds, val_labels, val_probs = validate(
        model, val_loader, criterion, device
    )
    
    # Update scheduler
    scheduler.step(val_loss)
    current_lr = optimizer.param_groups[0]['lr']
    
    # Log metrics
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    history['learning_rates'].append(current_lr)
    
    # Print epoch summary
    epoch_time = time.time() - epoch_start_time
    print(f"\nEpoch Summary:")
    print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
    print(f"  Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")
    print(f"  Learning Rate: {current_lr:.6f}")
    print(f"  Time: {epoch_time:.2f}s")
    
    # Save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_val_acc = val_acc
        patience_counter = 0
        
        # Save model
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': val_loss,
            'val_acc': val_acc,
            'config': CONFIG
        }, os.path.join(CONFIG['output_dir'], 'best_model.pth'))
        
        print(f"  ✓ New best model saved! (Val Loss: {val_loss:.4f})")
    else:
        patience_counter += 1
        print(f"  No improvement. Patience: {patience_counter}/{CONFIG['early_stopping_patience']}")
    
    # Early stopping
    if patience_counter >= CONFIG['early_stopping_patience']:
        print(f"\nEarly stopping triggered after {epoch+1} epochs")
        break
    
    # Check time limit (11.5 hours to be safe)
    elapsed_time = time.time() - start_time
    if elapsed_time > 11.5 * 3600:
        print(f"\nApproaching time limit. Stopping training.")
        break

total_time = time.time() - start_time
print(f"\n{'='*80}")
print(f"TRAINING COMPLETED")
print(f"Total time: {total_time/3600:.2f} hours")
print(f"Best validation loss: {best_val_loss:.4f}")
print(f"Best validation accuracy: {best_val_acc:.2f}%")
print(f"{'='*80}\n")

# Save training history
with open(os.path.join(CONFIG['output_dir'], 'training_history.json'), 'w') as f:
    json.dump(history, f, indent=2)


In [ ]:
print("Loading best model for evaluation...")
checkpoint = torch.load(os.path.join(CONFIG['output_dir'], 'best_model.pth'))
model.load_state_dict(checkpoint['model_state_dict'])
print(f"Loaded model from epoch {checkpoint['epoch']+1}")


In [ ]:
print("\n" + "="*80)
print("EVALUATING ON TEST SET")
print("="*80)

model.eval()
test_preds = []
test_labels = []
test_probs = []

with torch.no_grad():
    for inputs, labels in tqdm(test_loader, desc='Testing'):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        probs = torch.softmax(outputs, dim=1)
        _, predicted = outputs.max(1)
        
        test_preds.extend(predicted.cpu().numpy())
        test_labels.extend(labels.cpu().numpy())
        test_probs.extend(probs[:, 1].cpu().numpy())

test_preds = np.array(test_preds)
test_labels = np.array(test_labels)
test_probs = np.array(test_probs)

In [ ]:
print("\n" + "="*80)
print("1. EVALUATION METRICS")
print("="*80)

# Calculate metrics
accuracy = accuracy_score(test_labels, test_preds)
precision = precision_score(test_labels, test_preds, average='binary')
recall = recall_score(test_labels, test_preds, average='binary')
f1 = f1_score(test_labels, test_preds, average='binary')
auc_score = roc_auc_score(test_labels, test_probs)

print(f"\nTest Set Performance:")
print(f"  Accuracy:  {accuracy*100:.2f}%")
print(f"  Precision: {precision*100:.2f}%")
print(f"  Recall:    {recall*100:.2f}%")
print(f"  F1-Score:  {f1*100:.2f}%")
print(f"  AUC-ROC:   {auc_score:.4f}")

print("\nClassification Report:")
print(classification_report(test_labels, test_preds, 
                          target_names=['Real', 'Fake']))

# Confusion Matrix
cm = confusion_matrix(test_labels, test_preds)
print("\nConfusion Matrix:")
print(cm)


In [ ]:
print("\n" + "="*80)
print("2. EXPERIMENTAL METHODOLOGIES")
print("="*80)

methodology_report = f"""
Dataset:
  - Source: Kaggle 140k Real and Fake Faces (StyleGAN generated fakes)
  - Total Images: {len(train_data[0]) + len(val_data[0]) + len(test_data[0])}
  - Train Split: {CONFIG['train_split']*100}% ({len(train_data[0])} images)
  - Validation Split: {CONFIG['val_split']*100}% ({len(val_data[0])} images)
  - Test Split: {(1-CONFIG['train_split']-CONFIG['val_split'])*100}% ({len(test_data[0])} images)

Preprocessing Pipeline:
  1. Face Detection: Haar Cascade Classifier
  2. Face Cropping: Extract face region with 10% padding
  3. Image Rotation: Random rotation up to ±{CONFIG['rotation_degrees']}°
  4. Resize: {CONFIG['img_size']}x{CONFIG['img_size']} pixels
  5. Normalization: ImageNet statistics

Data Augmentation (Training only):
  - Random horizontal flip (p=0.5)
  - Random rotation (±{CONFIG['rotation_degrees']}°)
  - Color jitter (brightness, contrast, saturation ±0.2)

Model Architecture:
  - Type: Custom Convolutional Neural Network
  - Layers: 5 convolutional blocks + 3 FC layers
  - Parameters: {trainable_params:,}
  - Regularization: Batch normalization, dropout (0.2-0.5)
  - Activation: ReLU
  - Pooling: MaxPool2d + Global Average Pooling

Training Configuration:
  - Batch Size: {CONFIG['batch_size']}
  - Epochs: {CONFIG['num_epochs']} (early stopping enabled)
  - Optimizer: Adam
  - Learning Rate: {CONFIG['learning_rate']} (adaptive)
  - Weight Decay: {CONFIG['weight_decay']}
  - Loss Function: Cross Entropy
  - Device: {device} ({torch.cuda.device_count()} GPU(s))

Validation Strategy:
  - Early Stopping: Patience {CONFIG['early_stopping_patience']} epochs
  - LR Scheduler: ReduceLROnPlateau (patience {CONFIG['reduce_lr_patience']})
  - Best Model Selection: Lowest validation loss
"""

print(methodology_report)

In [ ]:
print("\n" + "="*80)
print("3. EXPERIMENTAL RESULTS AND PERFORMANCE ANALYSIS")
print("="*80)

results_analysis = f"""
Training Performance:
  - Final Training Accuracy: {history['train_acc'][-1]:.2f}%
  - Final Training Loss: {history['train_loss'][-1]:.4f}
  - Best Validation Accuracy: {best_val_acc:.2f}%
  - Best Validation Loss: {best_val_loss:.4f}
  - Training Time: {total_time/3600:.2f} hours
  - Epochs Completed: {len(history['train_loss'])}

Test Set Performance:
  - Test Accuracy: {accuracy*100:.2f}%
  - Test Precision: {precision*100:.2f}%
  - Test Recall: {recall*100:.2f}%
  - Test F1-Score: {f1*100:.2f}%
  - AUC-ROC Score: {auc_score:.4f}

Class-wise Performance:
  Real Images (Class 0):
    - True Positives: {cm[0,0]}
    - False Negatives: {cm[0,1]}
    - Precision: {cm[0,0]/(cm[0,0]+cm[1,0])*100:.2f}%
    - Recall: {cm[0,0]/(cm[0,0]+cm[0,1])*100:.2f}%
  
  Fake Images (Class 1):
    - True Positives: {cm[1,1]}
    - False Negatives: {cm[1,0]}
    - Precision: {cm[1,1]/(cm[1,1]+cm[0,1])*100:.2f}%
    - Recall: {cm[1,1]/(cm[1,1]+cm[1,0])*100:.2f}%

Model Generalization:
  - Train-Val Gap: {abs(history['train_acc'][-1] - history['val_acc'][-1]):.2f}%
  - Val-Test Gap: {abs(best_val_acc - accuracy*100):.2f}%
  - Overfitting Assessment: {'Minimal' if abs(history['train_acc'][-1] - accuracy*100) < 5 else 'Moderate' if abs(history['train_acc'][-1] - accuracy*100) < 10 else 'Significant'}
"""

print(results_analysis)

In [ ]:
print("\nGenerating visualizations...")

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")


In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Plot 1: Loss Curves
axes[0, 0].plot(history['train_loss'], label='Train Loss', linewidth=2, marker='o')
axes[0, 0].plot(history['val_loss'], label='Val Loss', linewidth=2, marker='s')
axes[0, 0].set_xlabel('Epoch', fontsize=12)
axes[0, 0].set_ylabel('Loss', fontsize=12)
axes[0, 0].set_title('Training and Validation Loss', fontsize=14, fontweight='bold')
axes[0, 0].legend(fontsize=10)
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Accuracy Curves
axes[0, 1].plot(history['train_acc'], label='Train Accuracy', linewidth=2, marker='o')
axes[0, 1].plot(history['val_acc'], label='Val Accuracy', linewidth=2, marker='s')
axes[0, 1].set_xlabel('Epoch', fontsize=12)
axes[0, 1].set_ylabel('Accuracy (%)', fontsize=12)
axes[0, 1].set_title('Training and Validation Accuracy', fontsize=14, fontweight='bold')
axes[0, 1].legend(fontsize=10)
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Learning Rate Schedule
axes[1, 0].plot(history['learning_rates'], linewidth=2, marker='o', color='red')
axes[1, 0].set_xlabel('Epoch', fontsize=12)
axes[1, 0].set_ylabel('Learning Rate', fontsize=12)
axes[1, 0].set_title('Learning Rate Schedule', fontsize=14, fontweight='bold')
axes[1, 0].set_yscale('log')
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Train-Val Loss Comparison
epochs = range(1, len(history['train_loss']) + 1)
axes[1, 1].fill_between(epochs, history['train_loss'], history['val_loss'], 
                         alpha=0.3, label='Loss Gap')
axes[1, 1].plot(history['train_loss'], label='Train Loss', linewidth=2)
axes[1, 1].plot(history['val_loss'], label='Val Loss', linewidth=2)
axes[1, 1].set_xlabel('Epoch', fontsize=12)
axes[1, 1].set_ylabel('Loss', fontsize=12)
axes[1, 1].set_title('Generalization Gap Analysis', fontsize=14, fontweight='bold')
axes[1, 1].legend(fontsize=10)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(CONFIG['output_dir'], 'learning_curves.png'), dpi=300, bbox_inches='tight')
plt.show()

print("Learning curves saved!")


In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True,
            xticklabels=['Real', 'Fake'],
            yticklabels=['Real', 'Fake'],
            ax=ax, annot_kws={'size': 16})
ax.set_xlabel('Predicted Label', fontsize=14, fontweight='bold')
ax.set_ylabel('True Label', fontsize=14, fontweight='bold')
ax.set_title('Confusion Matrix - Test Set', fontsize=16, fontweight='bold')

# Add percentages
for i in range(2):
    for j in range(2):
        percentage = cm[i, j] / cm[i].sum() * 100
        ax.text(j + 0.5, i + 0.7, f'({percentage:.1f}%)', 
                ha='center', va='center', fontsize=12, color='gray')

plt.tight_layout()
plt.savefig(os.path.join(CONFIG['output_dir'], 'confusion_matrix.png'), dpi=300, bbox_inches='tight')
plt.show()

print("Confusion matrix saved!")

In [ ]:
fpr, tpr, thresholds = roc_curve(test_labels, test_probs)

fig, ax = plt.subplots(figsize=(10, 8))
ax.plot(fpr, tpr, linewidth=3, label=f'ROC Curve (AUC = {auc_score:.4f})')
ax.plot([0, 1], [0, 1], 'k--', linewidth=2, label='Random Classifier')
ax.set_xlabel('False Positive Rate', fontsize=14, fontweight='bold')
ax.set_ylabel('True Positive Rate', fontsize=14, fontweight='bold')
ax.set_title('ROC Curve - Test Set', fontsize=16, fontweight='bold')
ax.legend(fontsize=12, loc='lower right')
ax.grid(True, alpha=0.3)
ax.set_xlim([0.0, 1.0])
ax.set_ylim([0.0, 1.05])

plt.tight_layout()
plt.savefig(os.path.join(CONFIG['output_dir'], 'roc_curve.png'), dpi=300, bbox_inches='tight')
plt.show()

print("ROC curve saved!")


In [ ]:
metrics_names = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
metrics_values = [accuracy * 100, precision * 100, recall * 100, f1 * 100]

fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.bar(metrics_names, metrics_values, color=['#3498db', '#2ecc71', '#e74c3c', '#f39c12'], 
              alpha=0.8, edgecolor='black', linewidth=1.5)

# Add value labels on bars
for bar, value in zip(bars, metrics_values):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{value:.2f}%',
            ha='center', va='bottom', fontsize=12, fontweight='bold')

ax.set_ylabel('Score (%)', fontsize=14, fontweight='bold')
ax.set_title('Test Set Performance Metrics', fontsize=16, fontweight='bold')
ax.set_ylim([0, 105])
ax.grid(True, axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(CONFIG['output_dir'], 'performance_metrics.png'), dpi=300, bbox_inches='tight')
plt.show()

print("Performance metrics chart saved!")

In [ ]:
print("\n" + "="*80)
print("FINAL SUMMARY REPORT")
print("="*80)

summary_report = f"""
╔══════════════════════════════════════════════════════════════════════════════╗
║                    AI-GENERATED FACE DETECTION MODEL                         ║
║                         Training Summary Report                              ║
╚══════════════════════════════════════════════════════════════════════════════╝

1. EVALUATION METRICS
{'='*80}
   Test Set Performance:
   ├─ Accuracy:  {accuracy*100:.2f}%
   ├─ Precision: {precision*100:.2f}%
   ├─ Recall:    {recall*100:.2f}%
   ├─ F1-Score:  {f1*100:.2f}%
   └─ AUC-ROC:   {auc_score:.4f}

   Confusion Matrix:
   ┌─────────────┬──────────┬──────────┐
   │             │ Pred: Real│ Pred: Fake│
   ├─────────────┼──────────┼──────────┤
   │ True: Real  │   {cm[0,0]:5d}  │   {cm[0,1]:5d}  │
   │ True: Fake  │   {cm[1,0]:5d}  │   {cm[1,1]:5d}  │
   └─────────────┴──────────┴──────────┘

2. EXPERIMENTAL METHODOLOGIES
{'='*80}
   Dataset Configuration:
   ├─ Total Samples: {len(train_data[0]) + len(val_data[0]) + len(test_data[0]):,}
   ├─ Train: {len(train_data[0]):,} ({CONFIG['train_split']*100:.0f}%)
   ├─ Validation: {len(val_data[0]):,} ({CONFIG['val_split']*100:.0f}%)
   └─ Test: {len(test_data[0]):,} ({(1-CONFIG['train_split']-CONFIG['val_split'])*100:.0f}%)

   Preprocessing Pipeline:
   ├─ Face Detection: Haar Cascade
   ├─ Face Cropping: Auto with 10% padding
   ├─ Rotation: ±{CONFIG['rotation_degrees']}°
   ├─ Resize: {CONFIG['img_size']}x{CONFIG['img_size']}
   └─ Normalization: ImageNet statistics

   Model Architecture:
   ├─ Type: Custom CNN (5 conv blocks + 3 FC)
   ├─ Total Parameters: {total_params:,}
   ├─ Trainable Parameters: {trainable_params:,}
   └─ Regularization: BatchNorm + Dropout

3. EXPERIMENTAL RESULTS AND PERFORMANCE ANALYSIS
{'='*80}
   Training Dynamics:
   ├─ Epochs Completed: {len(history['train_loss'])}
   ├─ Training Time: {total_time/3600:.2f} hours
   ├─ Final Train Accuracy: {history['train_acc'][-1]:.2f}%
   ├─ Best Val Accuracy: {best_val_acc:.2f}%
   └─ Test Accuracy: {accuracy*100:.2f}%

   Generalization Analysis:
   ├─ Train-Val Gap: {abs(history['train_acc'][-1] - history['val_acc'][-1]):.2f}%
   ├─ Val-Test Gap: {abs(best_val_acc - accuracy*100):.2f}%
   └─ Overfitting: {'Minimal ✓' if abs(history['train_acc'][-1] - accuracy*100) < 5 else 'Moderate ⚠' if abs(history['train_acc'][-1] - accuracy*100) < 10 else 'Significant ✗'}

   Class Performance:
   ├─ Real Images:
   │  ├─ Precision: {cm[0,0]/(cm[0,0]+cm[1,0])*100:.2f}%
   │  └─ Recall: {cm[0,0]/(cm[0,0]+cm[0,1])*100:.2f}%
   └─ Fake Images:
      ├─ Precision: {cm[1,1]/(cm[1,1]+cm[0,1])*100:.2f}%
      └─ Recall: {cm[1,1]/(cm[1,1]+cm[1,0])*100:.2f}%

4. OBJECTIVE AND LOSS FUNCTIONS
{'='*80}
   Loss Function: Cross Entropy Loss
   ├─ Purpose: Multi-class classification
   ├─ Formula: L = -Σ y_i * log(ŷ_i)
   └─ Properties: Penalizes confident wrong predictions

   Optimizer: Adam (Adaptive Moment Estimation)
   ├─ Learning Rate: {CONFIG['learning_rate']}
   ├─ Weight Decay: {CONFIG['weight_decay']} (L2 regularization)
   ├─ Beta1: 0.9 (default)
   └─ Beta2: 0.999 (default)

   Learning Rate Scheduler:
   ├─ Type: ReduceLROnPlateau
   ├─ Monitor: Validation Loss
   ├─ Patience: {CONFIG['reduce_lr_patience']} epochs
   ├─ Reduction Factor: {CONFIG['reduce_lr_factor']}
   └─ Final LR: {history['learning_rates'][-1]:.6f}

5. LEARNING CURVES
{'='*80}
   Training Loss Progression:
   ├─ Initial: {history['train_loss'][0]:.4f}
   ├─ Final: {history['train_loss'][-1]:.4f}
   └─ Reduction: {(1 - history['train_loss'][-1]/history['train_loss'][0])*100:.1f}%

   Validation Loss Progression:
   ├─ Initial: {history['val_loss'][0]:.4f}
   ├─ Best: {best_val_loss:.4f}
   └─ Final: {history['val_loss'][-1]:.4f}

   Accuracy Progression:
   ├─ Train: {history['train_acc'][0]:.2f}% → {history['train_acc'][-1]:.2f}%
   ├─ Val: {history['val_acc'][0]:.2f}% → {history['val_acc'][-1]:.2f}%
   └─ Test: {accuracy*100:.2f}%

6. HYPERPARAMETER CONFIGURATION
{'='*80}
   Model Hyperparameters:
   ├─ Input Size: {CONFIG['img_size']}x{CONFIG['img_size']}x3
   ├─ Dropout Rate: 0.5
   ├─ Batch Normalization: Enabled
   └─ Activation: ReLU

   Training Hyperparameters:
   ├─ Batch Size: {CONFIG['batch_size']}
   ├─ Max Epochs: {CONFIG['num_epochs']}
   ├─ Initial Learning Rate: {CONFIG['learning_rate']}
   ├─ Weight Decay: {CONFIG['weight_decay']}
   ├─ Early Stopping Patience: {CONFIG['early_stopping_patience']}
   └─ LR Reduction Patience: {CONFIG['reduce_lr_patience']}

   Data Augmentation:
   ├─ Random Rotation: ±{CONFIG['rotation_degrees']}°
   ├─ Horizontal Flip: 50% probability
   ├─ Color Jitter: ±0.2
   └─ Applied to: Training set only

   Computational Resources:
   ├─ Device: {device}
   ├─ GPUs: {torch.cuda.device_count()}
   ├─ GPU Model: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'N/A'}
   └─ Time Limit: 12 hours (Kaggle)

╔══════════════════════════════════════════════════════════════════════════════╗
║                           TRAINING COMPLETED                                 ║
║                                                                              ║
║  All models, logs, and visualizations saved to: {CONFIG['output_dir']}
║                                                                              ║
║  Generated Files:                                                            ║
║  ├─ best_model.pth           (Best model checkpoint)                        ║
║  ├─ training_history.json    (Training logs)                                ║
║  ├─ learning_curves.png      (Loss & accuracy plots)                        ║
║  ├─ confusion_matrix.png     (Confusion matrix heatmap)                     ║
║  ├─ roc_curve.png            (ROC curve)                                    ║
║  └─ performance_metrics.png  (Bar chart of metrics)                         ║
╚══════════════════════════════════════════════════════════════════════════════╝
"""

print(summary_report)

# Save summary report
with open(os.path.join(CONFIG['output_dir'], 'summary_report.txt'), 'w') as f:
    f.write(summary_report)

print("\n✓ Summary report saved to summary_report.txt")


In [ ]:
print("\nSaving predictions...")

# Create predictions dataframe
predictions_df = pd.DataFrame({
    'true_label': test_labels,
    'predicted_label': test_preds,
    'probability_fake': test_probs,
    'correct': test_labels == test_preds
})

predictions_df.to_csv(os.path.join(CONFIG['output_dir'], 'test_predictions.csv'), index=False)
print("✓ Test predictions saved to test_predictions.csv")


In [ ]:
print("\nExporting model for inference...")

# Save model architecture and weights separately for easier loading
torch.save(model.state_dict(), os.path.join(CONFIG['output_dir'], 'model_weights.pth'))

# Save complete model info
model_info = {
    'architecture': 'FaceClassifierCNN',
    'input_size': CONFIG['img_size'],
    'num_classes': 2,
    'total_parameters': total_params,
    'trainable_parameters': trainable_params,
    'best_val_accuracy': best_val_acc,
    'test_accuracy': accuracy * 100,
    'training_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'config': CONFIG
}

with open(os.path.join(CONFIG['output_dir'], 'model_info.json'), 'w') as f:
    json.dump(model_info, f, indent=2)

print("✓ Model weights saved to model_weights.pth")
print("✓ Model info saved to model_info.json")

In [ ]:
print("\n" + "="*80)
print("ALL TASKS COMPLETED SUCCESSFULLY! 🎉")
print("="*80)
print(f"\nFinal Test Accuracy: {accuracy*100:.2f}%")
print(f"Total Training Time: {total_time/3600:.2f} hours")
print(f"\nAll outputs saved to: {CONFIG['output_dir']}")
print("\nYou can now:")
print("  1. Review the learning curves and metrics visualizations")
print("  2. Load the best model for inference")
print("  3. Analyze the test predictions")
print("  4. Fine-tune hyperparameters based on the results")
print("\n" + "="*80)